In [14]:
import torch
import numpy as np
import torch
import torchvision as torchvision
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
from torch import nn
from torch.nn.functional import relu
# Load the model file
model_weights = torch.load(r'C:\Users\arsha\PycharmProjects\DSU\brestcancer_detection\breast_cancer_diagnosis.pt', map_location=torch.device('cpu'),weights_only=True)

# Check if it's a state_dict (weights only)
if isinstance(model_weights, dict):
    # If it contains keys like "conv1.weight", "fc.bias", it's likely a state dict
    print("This file contains only weights (state_dict).")
    for key, value in model_weights.items():
        print(key, value.shape)
else:
    print("This file contains the entire model.")

This file contains only weights (state_dict).
conv1.weight torch.Size([6, 3, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([16, 6, 5, 5])
conv2.bias torch.Size([16])
fc1.weight torch.Size([64, 400])
fc1.bias torch.Size([64])
fc2.weight torch.Size([1, 64])
fc2.bias torch.Size([1])


In [15]:


class BreastCancerModel(nn.Module):
    def __init__(self):
        super(BreastCancerModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=(5, 5))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5, 5))
        self.pool2 = nn.AdaptiveAvgPool2d(output_size=(5, 5))
        self.fc1 = nn.Linear(16 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool1(relu(self.conv1(x)))
        x = self.pool2(relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x)

# Load the model and weights
model = BreastCancerModel()
model.load_state_dict(torch.load(r'C:\Users\arsha\PycharmProjects\DSU\brestcancer_detection\breast_cancer_diagnosis.pt', map_location=torch.device('cpu'),weights_only=True), strict=False)
model.eval()



transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 128x128 or your preferred size
    transforms.ToTensor()             # Convert image to tensor
])

# Define the paths to your image folders
image_folders = {
    'normal': r'C:\Users\arsha\PycharmProjects\DSU\brestcancer_detection\normal',
    'benign': r"C:\Users\arsha\PycharmProjects\DSU\brestcancer_detection\benign",
    'malignant': r'C:\Users\arsha\PycharmProjects\DSU\brestcancer_detection\malignant'
}

images = []
labels = []  # To store labels corresponding to images

# Loop through each folder and load images and labels
for label, folder in enumerate(image_folders.values()):
    for filename in os.listdir(folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            image = Image.open(img_path).convert('RGB')  # Ensure it's in RGB format
            image = transform(image)  # Apply transformations
            
            images.append(image)  # Append the image to the list
            
            # Assign label based on folder
            if label == 0:  # Normal
                labels.append(0)  # No cancer
            elif label == 1:  # Benign
                labels.append(1)  # or 1 if you consider benign as cancerous
            elif label == 2:  # Malignant
                labels.append(1)  # Positive for cancer

# Convert lists to tensors
input_images= torch.stack(images)  
input_labels = torch.tensor(labels, dtype=torch.float32)  # Shape: [N]

# Check shapes for debugging
print(f'Images tensor shape: {input_images.shape}')
print(f'Labels tensor shape: {input_labels.shape}')


# Make predictions
with torch.no_grad():
    outputs = model(input_images)
    predictions = (outputs > 0.5).float()

# Calculate accuracy
correct_predictions = (predictions.squeeze() == input_labels).sum().item()
total_predictions = input_labels.size(0)
accuracy = correct_predictions / total_predictions * 100

print(f'Accuracy: {accuracy:.2f}%')


Images tensor shape: torch.Size([1578, 3, 128, 128])
Labels tensor shape: torch.Size([1578])
Accuracy: 50+16.86%
